## Extract Transform Load example

**Sample data: Watson Analytics Sample Data – Sales Products**<br>
https://www.ibm.com/communities/analytics/watson-analytics-blog/sales-products-sample-data/

In [1]:
import sqlite3
from pathlib import Path
import pandas as pd

#### Extract data from csv

In [2]:
df = pd.DataFrame.from_csv("WA_Sales_Products_2012_Q4.csv", index_col=None)

In [3]:
df.head()

,Retailer country,Order method type,Retailer type,Product line,Product type,Product,Year,Quarter,Revenue,Quantity,Gross margin
0,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Water Bag,2012,Q4 2012,21076.95,3405,0.526656
1,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,2012,Q4 2012,64262.38,527,0.347548
2,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Single Flame,2012,Q4 2012,51086.64,814,0.260994
3,United States,Fax,Outdoors Shop,Camping Equipment,Sleeping Bags,Hibernator,2012,Q4 2012,72598.52,526,0.376902
4,United States,Fax,Outdoors Shop,Camping Equipment,Lanterns,Firefly Lite,2012,Q4 2012,9887.70,690,0.528960


#### Transform data

In [4]:
# Get from the Quarter column only the quarter information (e.g. from "Q4 2012" --> "Q4")
df['Quarter'] = df['Quarter'].astype(str).str[:2]

In [5]:
df.head()

,Retailer country,Order method type,Retailer type,Product line,Product type,Product,Year,Quarter,Revenue,Quantity,Gross margin
0,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Water Bag,2012,Q4,21076.95,3405,0.526656
1,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,2012,Q4,64262.38,527,0.347548
2,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Single Flame,2012,Q4,51086.64,814,0.260994
3,United States,Fax,Outdoors Shop,Camping Equipment,Sleeping Bags,Hibernator,2012,Q4,72598.52,526,0.376902
4,United States,Fax,Outdoors Shop,Camping Equipment,Lanterns,Firefly Lite,2012,Q4,9887.70,690,0.528960


In [6]:
# Remove space and format in lowercase the column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [7]:
df.head()

,retailer_country,order_method_type,retailer_type,product_line,product_type,product,year,quarter,revenue,quantity,gross_margin
0,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Water Bag,2012,Q4,21076.95,3405,0.526656
1,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,2012,Q4,64262.38,527,0.347548
2,United States,Fax,Outdoors Shop,Camping Equipment,Cooking Gear,TrailChef Single Flame,2012,Q4,51086.64,814,0.260994
3,United States,Fax,Outdoors Shop,Camping Equipment,Sleeping Bags,Hibernator,2012,Q4,72598.52,526,0.376902
4,United States,Fax,Outdoors Shop,Camping Equipment,Lanterns,Firefly Lite,2012,Q4,9887.70,690,0.528960


In [8]:
output_columns = ['year', 'quarter', 'retailer_country', 'retailer_type','order_method_type', 'revenue']

# Keep only the columns that match the target table
df = df[df.columns[df.columns.isin(output_columns)]]

In [9]:
df.head()

,retailer_country,order_method_type,retailer_type,year,quarter,revenue
0,United States,Fax,Outdoors Shop,2012,Q4,21076.95
1,United States,Fax,Outdoors Shop,2012,Q4,64262.38
2,United States,Fax,Outdoors Shop,2012,Q4,51086.64
3,United States,Fax,Outdoors Shop,2012,Q4,72598.52
4,United States,Fax,Outdoors Shop,2012,Q4,9887.70


In [10]:
 # Sum revenues grouping by output_columns
df = df[output_columns].groupby(by=['year', 'quarter', 'retailer_country', 'retailer_type', 'order_method_type'], 
                                as_index=False).sum()

In [11]:
df.head()

,year,quarter,retailer_country,retailer_type,order_method_type,revenue
0,2012,Q4,Australia,Department Store,Telephone,725829.34
1,2012,Q4,Australia,Department Store,Web,1324205.94
2,2012,Q4,Australia,Direct Marketing,Sales visit,90599.11
3,2012,Q4,Australia,Equipment Rental Store,Sales visit,82976.95
4,2012,Q4,Australia,Eyewear Store,Sales visit,120849.37


In [12]:
# Remove decimals
df['revenue'] = df['revenue'].apply(lambda x: '{:.0f}'.format(x))

In [13]:
df.head()

,year,quarter,retailer_country,retailer_type,order_method_type,revenue
0,2012,Q4,Australia,Department Store,Telephone,725829
1,2012,Q4,Australia,Department Store,Web,1324206
2,2012,Q4,Australia,Direct Marketing,Sales visit,90599
3,2012,Q4,Australia,Equipment Rental Store,Sales visit,82977
4,2012,Q4,Australia,Eyewear Store,Sales visit,120849


#### Create SQLite revenues table where to store the data

In [14]:
# Connect to SQLlite DB
cwd = Path().resolve()
print("Where the db is stored: {}/revenues.db".format(cwd))

# Connect and create DB if it doesn't exist
conn = sqlite3.connect(str(cwd / 'revenues.db'))
cursor = conn.cursor()

Where the db is stored: /Users/enrica.pasqua/Documents/revenues.db


In [15]:
# Create revenues table (and delete if already exists)

drop_table = "DROP TABLE IF EXISTS revenues;"
cursor.execute(drop_table)

ddl_table = """
            CREATE TABLE revenues(
                year INTEGER,
                quarter	INTEGER, 
                retailer_country TEXT,
                retailer_type TEXT,	
                order_method_type TEXT,
                revenue INTEGER)
            ;"""

cursor.execute(ddl_table)
conn.commit()

#### Load data in table

In [16]:
# insert into table via Pandas
df.to_sql('revenues', conn, if_exists='append', index=False)

In [17]:
# Verify data are in the table
loaded_data = cursor.execute('SELECT * FROM revenues;').fetchall()
conn.close()

In [18]:
print("rows in the table: ", len(loaded_data))

rows in the table:  231


In [19]:
for row in loaded_data[:5]:
    print(row)

(2012, 'Q4', 'Australia', 'Department Store', 'Telephone', 725829)
(2012, 'Q4', 'Australia', 'Department Store', 'Web', 1324206)
(2012, 'Q4', 'Australia', 'Direct Marketing', 'Sales visit', 90599)
(2012, 'Q4', 'Australia', 'Equipment Rental Store', 'Sales visit', 82977)
(2012, 'Q4', 'Australia', 'Eyewear Store', 'Sales visit', 120849)
